In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openmim
!mim install mmengine
# !pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.4.0/index.html


In [ ]:
!git clone https://github.com/open-mmlab/mmaction2.git
!cd mmaction2
!pip install -r requirements.txt
!pip install -v -e .

fatal: destination path 'mmaction2' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Using pip 24.3.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200616-e9b6ebf5.pth -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb.pth
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_50e_sthv2_rgb/tsn_r50_1x1x3_50e_sthv2_rgb_20210324-e8e5b019.pth -O checkpoints/tsn_r50_1x1x3_50e_sthv2_rgb.pth
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_inference/tsn_r50_video_inference_20210129-e9b1bc5d.pth -O checkpoints/tsn_r50_video_inference.pth
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_3d_1x1x3_50e_kinetics400_flow/tsn_r50_3d_1x1x3_50e_kinetics400_flow_20200616-7dfc7b8a.pth -O checkpoints/tsn_r50_3d_1x1x3_50e_kinetics400_flow.pth
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_3d_1x1x3_50e_kinetics400_rgb/tsn_r50_3d_1x1x3_50e_kinetics400_rgb_20200616-1c7b490d.pth -O checkpoints/tsn_r50_3d_1x1x3_50e_kinetics400_rgb.pth
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r101_1x1x3_100e_kinetics400_rgb/tsn_r101_1x1x3_100e_kinetics400_rgb_20200616-b7ecfe70.pth -O checkpoints/tsn_r101_1x1x3_100e_kinetics400_rgb.pth

# Download TSN R50 model trained on Kinetics-400 RGB dataset
!wget https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth -P checkpoints/

# Download TSN R50 model trained on HMDB51 RGB dataset
!wget https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x8_50e_hmdb51_imagenet_rgb/tsn_r50_1x1x8_50e_hmdb51_imagenet_rgb_20201025-0c3b1fdc.pth -P checkpoints/

# Download TSN R50 model trained on UCF101 RGB dataset
!wget https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x8_50e_ucf101_imagenet_rgb/tsn_r50_1x1x8_50e_ucf101_imagenet_rgb_20201025-6f8c0cce.pth -P checkpoints/


--2024-12-28 17:24:01--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200616-e9b6ebf5.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.82.164, 163.181.82.169, 163.181.82.165, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.82.164|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-28 17:24:03 ERROR 404: Not Found.

--2024-12-28 17:24:03--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_50e_sthv2_rgb/tsn_r50_1x1x3_50e_sthv2_rgb_20210324-e8e5b019.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.82.164, 163.181.82.169, 163.181.82.165, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.82.164|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-28 17:24:04 ERROR 404: Not Found.

--2024-12-28 17:24:04--  https://download.openmmlab.com/

In [ ]:
from mmengine import Config

# Load the updated TSN config file
config_file = 'mmaction2/configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py'
cfg = Config.fromfile(config_file)

# Update train dataset configuration
cfg.train_dataloader.dataset = dict(
    type='RawframeDataset',
    modality='RGB',
    ann_file='/content/drive/MyDrive/mmaction/annotations/train.txt',
    data_prefix={'img': '/content/drive/MyDrive/mmaction/videos/'},  # Corrected data_prefix to match actual file structure
    pipeline=[
        dict(type='SampleFrames', clip_len=5, frame_interval=1, num_clips=1),
        dict(type='RawFrameDecode', io_backend='disk'),
        dict(type='Resize', scale=(-1, 256)),
        dict(type='CenterCrop', crop_size=224),
        dict(type='FormatShape', input_format='NCHW'),  # Ensure channels-first format
        dict(type='PackActionInputs', meta_keys=['img_shape', 'label'])  # Package inputs
    ],
    test_mode=False
)

# Update validation dataset configuration
cfg.val_dataloader.dataset = dict(
    type='RawframeDataset',
    modality='RGB',
    ann_file='/content/drive/MyDrive/mmaction/annotations/val.txt',
    data_prefix={'img': '/content/drive/MyDrive/mmaction/videos/'},  # Corrected data_prefix to match actual file structure
    pipeline=[
        dict(type='SampleFrames', clip_len=5, frame_interval=1, num_clips=1),
        dict(type='RawFrameDecode', io_backend='disk'),
        dict(type='Resize', scale=(-1, 256)),
        dict(type='CenterCrop', crop_size=224),
        dict(type='FormatShape', input_format='NCHW'),  # Ensure channels-first format
        dict(type='PackActionInputs', meta_keys=['img_shape', 'label'])  # Package inputs
    ],
    test_mode=True
)

cfg.model.data_preprocessor = dict(
    type='ActionDataPreprocessor',
    mean=[123.675, 116.28, 103.53],  # RGB mean values
    std=[58.395, 57.12, 57.375],  # RGB std values
    format_shape='NCHW'  # Channels-first format
)


# Update model configuration
cfg.model.cls_head.num_classes = 2  # Strike and neutral classes
cfg.train_cfg.max_epochs = 20  # Set maximum epochs for training

# Update working directory
cfg.work_dir = '/content/mmaction_work_dir'

# Save the modified config
custom_config_path = '/content/mmaction2/tsn_custom_config.py'
cfg.dump(custom_config_path)
print(f"Custom configuration file saved to: {custom_config_path}")


Custom configuration file saved to: /content/mmaction2/tsn_custom_config.py


In [ ]:
!pwd

/content


In [ ]:
!python mmaction2/tools/train.py /content/mmaction2/tsn_custom_config.py --work-dir /content/mmaction_work_dir

Streaming output truncated to the last 5000 lines.
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(BELOW_NORMAL) LoggerHook                         
(LOW         ) ParamSchedulerHook                 
(VERY_LOW    ) CheckpointHook                     
 --------

In [ ]:
import os
import cv2
import torch
from mmaction.structures import ActionDataSample
from moviepy.editor import ImageSequenceClip
from torchvision.transforms import ToTensor
from mmaction.apis import init_recognizer, inference_recognizer

# Declare class names directly
class_names = {0: "miss", 1: "strike"}


# Paths
video_path = "/content/drive/MyDrive/test_file.mp4"
gif_path = "/content/drive/MyDrive/test.gif"
output_frames_dir = "/content/frames"
os.makedirs(output_frames_dir, exist_ok=True)

# Parameters
sequence_length = 5
config_file = "/content/mmaction_work_dir/tsn_custom_config.py"  # Path to your MMAction2 config
model_checkpoint = '/content/mmaction_work_dir/epoch_20.pth'  # Replace with your model's checkpoint path


# Initialize the recognizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = init_recognizer(config_file, checkpoint_file, device=device)

# Extract first 2 seconds from the video using OpenCV
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 2 * fps  # First 2 seconds
frame_idx = 0

frames = []
while cap.isOpened() and frame_idx < frame_count:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
    frame_idx += 1

cap.release()

# Save individual frames
frame_paths = []
for i, frame in enumerate(frames):
    frame_path = os.path.join(output_frames_dir, f"frame_{i:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_paths.append(frame_path)

# Prepare sequences of 5 frames
sequence_length = 5
sequences = [frames[i:i + sequence_length] for i in range(0, len(frames), sequence_length) if len(frames[i:i + sequence_length]) == sequence_length]

# Perform inference for each sequence
predicted_labels = []
for sequence in sequences:
    # Save the sequence temporarily as a video
    temp_video_path = "/content/temp_sequence.mp4"
    out = cv2.VideoWriter(temp_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (sequence[0].shape[1], sequence[0].shape[0]))
    for frame in sequence:
        out.write(frame)
    out.release()

    # Perform inference
    results = inference_recognizer(model, temp_video_path)

    # Extract the prediction scores
    scores = results.get('pred_score', None)
    if scores is None:
        raise ValueError("No prediction scores found in the result object.")

    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()

    # Map the highest score index to class label
    top_pred_idx = scores.argmax()
    top_label = class_names[top_pred_idx]  # Use the declared class names
    predicted_labels.append(top_label)

# Overlay labels on frames
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (0, 255, 0)  # Green
thickness = 2

for i, sequence in enumerate(sequences):
    label = predicted_labels[i]
    for j, frame in enumerate(sequence):
        frame_idx = i * sequence_length + j
        text = f"Label: {label}"
        cv2.putText(frame, text, (10, 30), font, font_scale, font_color, thickness)
        frame_paths[frame_idx] = os.path.join(output_frames_dir, f"frame_{frame_idx:04d}_labeled.jpg")
        cv2.imwrite(frame_paths[frame_idx], frame)

# Create GIF
clip = ImageSequenceClip([cv2.imread(path) for path in frame_paths], fps=fps)
clip.write_gif(gif_path, fps=fps)

print(f"GIF saved at {gif_path}")

  checkpoint = torch.load(filename, map_location=map_location)



Loads checkpoint by local backend from path: /content/mmaction_work_dir/epoch_20.pth
MoviePy - Building file /content/drive/MyDrive/test.gif with imageio.


GIF saved at /content/drive/MyDrive/test.gif
